In [9]:
import torch 
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from process import *
from drawbox import *
from framedif import *

import os
import sys
sys.path.append('../va_pipeline/')

%matplotlib inline


In [27]:
X_RES = 1920
Y_RES = 1080
cap = cv2.VideoCapture('../samples/medium.mp4')
out = cv2.VideoWriter(
    './test.mp4', 
    cv2.VideoWriter_fourcc(*'mp4v'), 
    30,
    (X_RES, Y_RES)
    )

model = torch.hub.load('ultralytics/yolov5', 'yolov5n')

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:  # no more frames to read
        break

    edge = get_frame_feature(frame, 5, 1, 100, 200)
    x,y,w,h = cv2.boundingRect(edge)

    cropped = frame[y:(y + h), x:(x + w)]
    cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

    output = model(cropped).pandas().xywh[0]
    output['xcenter'] += x
    output['ycenter'] += y

    for _, row in output.iterrows():
        x_center, y_center, width, height = row['xcenter'], row['ycenter'], row['width'], row['height']

        # Get top left corner coordinates
        topLeft = (int(x_center - width/2), int(y_center - height/2))
        bottomRight = (int(x_center + width/2), int(y_center + height/2))

        # Draw bounding box
        cv2.rectangle(frame, topLeft, bottomRight, (255, 0, 255), 2)

    out.write(frame)

cap.release()
out.release()

Using cache found in C:\Users\adamf/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-6-26 Python-3.9.13 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


In [28]:
X_RES = 1920
Y_RES = 1080
cap = cv2.VideoCapture('../samples/medium.mp4')
out = cv2.VideoWriter(
    './test_ground.mp4', 
    cv2.VideoWriter_fourcc(*'mp4v'), 
    30,
    (X_RES, Y_RES)
    )

model = torch.hub.load('ultralytics/yolov5', 'yolov5n')

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:  # no more frames to read
        break
    
    output = model(frame).pandas().xywh[0]
    
    for _, row in output.iterrows():
        x_center, y_center, width, height = row['xcenter'], row['ycenter'], row['width'], row['height']

        # Get top left corner coordinates
        topLeft = (int(x_center - width/2), int(y_center - height/2))
        bottomRight = (int(x_center + width/2), int(y_center + height/2))

        # Draw bounding box
        cv2.rectangle(frame, topLeft, bottomRight, (255, 0, 255), 2)

    out.write(frame)
    
    
cap.release()
out.release()

Using cache found in C:\Users\adamf/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-6-26 Python-3.9.13 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 
